In [22]:
# Mnist 手写数字图形识别
from __future__ import print_function
import os 
import struct 
import numpy as np 
import cv2
# 加载mnist
def load_mnist(path,kind='train'):
    labels_path = os.path.join(path,'%s-labels.idx1-ubyte' % kind)
    images_path = os.path.join(path,'%s-images.idx3-ubyte' % kind)
    with open(labels_path,'rb') as lbpath:
        magic,n = struct.unpack('>II', lbpath.read(8))
        labels= np.fromfile(lbpath,dtype=np.uint8)
    with open(images_path,'rb') as imgpath:
        magic,num,rows,cols= struct.unpack(">IIII", imgpath.read(16))
        images = np.fromfile(imgpath,dtype=np.uint8).reshape(len(labels),rows*cols)
    return images,labels

x_train, y_train = load_mnist('./data', kind='train')
print('Rows:%d,columns:%d' % (x_train.shape[0],x_train.shape[1]))
x_test, y_test = load_mnist('./data', kind='t10k')
print('Rows: %d, columns: %d' % (x_test.shape[0], x_test.shape[1]))

image1 = x_train[1].astype('float32').reshape(28,28,1)
cv2.imwrite('./data/1.jpg',image1)

img = cv2.imread('./data/1.jpg')
cv2.imshow("0",img)
cv2.waitKey(10000)


Rows:60000,columns:784
Rows: 10000, columns: 784


-1

In [20]:
import torch 
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import matplotlib.pyplot as plt 
from torchvision.datasets import MNIST 
from torchvision import transforms
from torch import nn 
import os 
from PIL import Image
import numpy as np 
import torch.nn.functional as F 



# 初始化数据内容和标签
def __init__(self,Data,Label):
    self.Data = Data
    self.Label = Label
    
# 获取数据内容和标签
def __getItem__(self,idx):
    data = torch.Tensor(self.Data[idx])
    label = torch.Tensor(self.Label[idx])

# 获取数据集大小
def __len__(self):
    return len(self.Data)

# 数据加载类 DataLoader

# 数据预处理
transform = transforms.Compose({
  transforms.ToTensor(),
  transforms.Normalize((0.1307,),(0.3081))
})

# 加载数据集
train_data = MNIST(root='./data',train=True,download=False,transform=transforms.ToTensor())
train_loader = DataLoader(train_data,shuffle=True,batch_size=64)

# 测试数据集
test_data = MNIST(root='./data',train=False,download=False,transform=transforms.ToTensor())
test_loader = DataLoader(test_data,shuffle=False,batch_size=64)
print(train_data)
print(test_data)


class Model(nn.Module): # 模型构建
    def __init__(self):
        super(Model,self).__init__()
        self.conv1 = nn.Conv2d(1,32,kernel_size=(5,5),stride=(1,1))
        self.conv2 = nn.Conv2d(32,64,kernel_size=(5,5),stride=(1,1))
        self.conv2_drop = nn.Dropout2d(p=0.5)
        self.fc1 = nn.Linear(1024,100)
        self.fc2 = nn.Linear(100,10)
        
    def forward(self,x):
        # relu(Rectified Linear Unit) 修正线性单元  
        # 当X <=0 时，relu = 0 , 当x >0 时，relu = x ;
        x = F.relu(F.max_pool2d(self.conv1(x),2))
        x= F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)),2))
        x = x.view(-1,1024)
        x = F.relu(self.fc1(x))
        x = F.dropout(x,training=self.training)
        x = self.fc2(x)
        return x

# 损失函数 交叉熵损失函数 CrossEntropyLoss
# 分为三步  1. Softmax  2. Log  3.NllLoss 

# Softmax回归是一个线性多分类模型，softmax 最终会给出预测值对于10个类别（0-9）出现的概率
# 最终模型的预测结果就是概率最大的类别，Softmax 计算公式如下
model = Model()
print(model)
# 计算交叉熵损失，相当于 softmax+log+nllloss
criterion = nn.CrossEntropyLoss()

# 优化器 SGD 
# 第一个参数是初始化参数值，第二个参数是学习率
optimizer = torch.optim.SGD(model.parameters(),lr=0.01,momentum=0.5)

# 模型训练
def train():
    for index,data in enumerate(train_loader):
        input,target = data # input 作为输入数据，target 作为标签
        optimizer.zero_grad()   # 梯度清零
        y_predict = model(input)    # 模型预测
        loss = criterion(y_predict,target)    # 计算损失
        loss.backward()  # 反向传播
        optimizer.step()  # 更新参数
        if index % 100 ==0:  # 每训练100次，保存模型
            torch.save(model.state_dict(),"./model/model.pkl") # 保存模型
            torch.save(optimizer.state_dict(),"./model/optimizer.pkl")  # 保存优化器
            print("损失值：%.2f" % loss.item())

# 加载模型 
if os.path.exists('./model/model.pkl'):
    model.load_state_dict(torch.load("./model/model.pkl"))
# 模型测试
def test():
    correct = 0  # 定义正确模型个数
    total = 0 # 总数
    with torch.no_grad(): # 测试不进行梯度计算
        for data in test_loader:
            input,target = data
            output = model(input) # output输出10个预测取值，其中最大的即为预测的数
            probability,predict = torch.max(output.data,dim=1) # 返回一个元组，第一个为最大概率值，第二个为最大概率值的下标
            total +=target.size(0) # target是形状为(batch_size,1)的矩阵，使用size(0)取出该批的大小 
            correct += (predict == target).sum().item() # predict和target均为(batch_size,1)的矩阵ranh ，sum()求出相等的个数
        print("准确率为：%.2f" % (correct / total))

def test_mydata():
    image = Image.open('./data/1.jpg') # 读取自定义的手写图片
    image = image.resize(28,28) # 裁剪尺寸为 28*28 
    image = image.convert('L') # 转换为灰度图像 
    transform = transforms.toTensor() 
    image = transform(image)
    image = image.resize(1,1,28,28)
    output = model(image)
    probability,predict = torch.max(output.data,dim=1)
    print('此手写图片值为%d,其最大的概率为：%.2f' % (predict[0],probability))
    plt.title('此手写图片值为：{}'.format((int(predict))),fontname='SimHei')
    plt.imshow(image.squeeze())
    plt.show()
        
if __name__ == '__main__':
    # test_mydata()
    for i in range(5):# 训练和测试进行五轮
        train()
        test()

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: ToTensor()
Dataset MNIST
    Number of datapoints: 10000
    Root location: ./data
    Split: Test
    StandardTransform
Transform: ToTensor()
Model(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (conv2_drop): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=1024, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=10, bias=True)
)
损失值：0.08
损失值：0.12
损失值：0.08
损失值：0.13
损失值：0.09
损失值：0.04
损失值：0.04
损失值：0.20
损失值：0.08
损失值：0.20
准确率为：0.98
损失值：0.10
损失值：0.12
损失值：0.04
损失值：0.10
损失值：0.10
损失值：0.10
损失值：0.03
损失值：0.09
损失值：0.11
损失值：0.08
准确率为：0.98
损失值：0.07
损失值：0.04
损失值：0.19
损失值：0.07
损失值：0.04
损失值：0.06
损失值：0.03
损失值：0.07
损失值：0.27
损失值：0.19
准确率为：0.98
损失值：0.03
损失值：0.08
损失值：0.03
损失值：0.06
损失值：0.04
损失值：0.12
损失值：0.03
损失值：0.04
损失值：0.04
损失值：0.03
准确率为：0.98
损失值：0.33
损失值：0.01
损失值：0.04
损失值：0.